# Imports

In [1]:
import pandas as pd
import numpy as np
from random import shuffle
from tqdm.notebook import tqdm
import time
from math import ceil

# Random dataset

In [8]:
np.random.seed(42)

In [9]:
nodes = 4
# creating a random matrix with respective probabilities
val = np.random.choice(2, size=(nodes, nodes), p=[0.7, 0.3])
val

array([[0, 1, 1, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 1],
       [1, 0, 0, 0]])

In [ ]:
# convert the random matrix to a adjacency matrix by make it symmetric to diagonal
for i, row in enumerate(val):
  val[:, i] = row
np.fill_diagonal(val, 1)
data = val

# Utility functions

In [2]:
def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(size))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes

In [3]:
def create_hash(minhash_fn, vector: list):
    # use this function for creating our signatures (eg the matching)
    signature = []
    for func in minhash_fn:
        for i in range(len(vector)):
            idx = func.index(i)
            signature_val = vector[idx]
            if signature_val == 1:
                signature.append(idx)
                break
    return signature

In [4]:
def load_dataset(ds_name):
    # read dataset and turn to np array
    data = pd.read_excel(ds_name)
    data = np.array(data)

    # dissimlar edges as no edge
    data[data == -1] = 0

    data = data.astype(int)

    return data

# Score Functions

In [5]:
def fast_score(data, cluster):
    score = 0
    nodes = len(data)
    for i in (range(nodes)):
        for j in range(i + 1, nodes):
            # if i and j are similar and not in a same cluster
            if data[i][j] == 1:
                # and not in the same cluster
                if cluster[i] != cluster[j]: score -= 1
            # else if they are not similar
            else:
                # and in the same cluster
                if cluster[i] == cluster[j]: score -= 1

    return score
             

In [6]:
def find_best_cluster(bands_clusters):
  max = -float('inf')
  max_cluster= None
  for cluster in bands_clusters:
    # scr = score(data, cluster)
    scr = fast_score(data, cluster)
    if scr > max:
      max = scr
      max_cluster = cluster
    if scr == 0:
      break
  return max_cluster, max

# Lsh functions

In [51]:
def LSH_simmilarity_matrix_permutation_arr(num_iter, data, row_num, band_num):
    #initialize
    best_cluster = None
    best_score = -float('inf')
    nodes = len(data)

    # num of iterations
    for _ in tqdm(range(num_iter)):
        
        # a perm. of size n
        permutation = np.arange(nodes).astype(int)
        shuffle(permutation)

        # new matrix with shuffled nodes
        shuff_data = []
        for i in range(nodes):
            shuff_data.append([data[i][j] for j in permutation])
        
        

        # simmilarity matrix is the new signiture
        sigs = np.array(shuff_data)

        # for each band
        bands_clusters = []
        for i in (range(band_num)):
            band_cluster = [0] * nodes

            # for each signiture calculate lsh and corresponding cluster
            for j, sig in enumerate(sigs):
                
                lsh = int(''.join(map(str,sig[i*row_num: (i+1)*row_num])), 2) % nodes
                
                # band_cluster[lsh] = (band_cluster.get(lsh,list()))+ [j]
                band_cluster[j] = lsh
            
            # add cluster to clusters
            bands_clusters.append(band_cluster)
        # print('claculating score')
        cluster, scr = find_best_cluster(bands_clusters)
        # print('score:', scr)
        if scr > best_score:
            best_score = scr
            best_cluster = cluster
        # print('best cluster:', best_cluster)
    return best_score , best_cluster
    # return best_score

    

In [8]:
def regular_LSH(num_iter, data, row_num, band_num, sigs_num):
  #initialize
  best_cluster = None
  best_score = -float('inf')
  nodes = len(data)

  for _ in tqdm(range(num_iter)):
    # we create 20 minhash vectors
    minhash_funcs = build_minhash_func(len(data), sigs_num)

    # now create signatures
    sigs = []
    for i in range(len(data)):
      sigs.append(create_hash(minhash_funcs, data[i]))

    bands_clusters = []
    for i in range(band_num):
      band_cluster = [0] * nodes
      for j, sig in enumerate(sigs):
        try:
          lsh = int(''.join(map(str,sig[i*row_num: (i+1)*row_num])), 10) % nodes
        except:
          print(''.join(map(str,sig[i*row_num: (i+1)*row_num])))
        band_cluster[j] = lsh
      bands_clusters.append(band_cluster)
    cluster, scr = find_best_cluster(bands_clusters)
    if scr > best_score:
      best_score = scr
      best_cluster = cluster
  return best_score, best_cluster


In [9]:
def LSH_singular_diff(num_iter, data, row_num, band_num, treshold, is_log = True):
    
    nodes = len(data) # number of nodes in dataset
    best_cluster = [0] * nodes
    single_nodes = [] # single nodes that have less similarity than treshold
    for i, row in enumerate(data):
        if sum(row) < treshold: # number of 1's in a row
            best_cluster[i] = nodes + i    # new cluster not involve in other clusters because of node++ index
            single_nodes.append(i)
    # print(data, single_nodes)

    # delete single elements from data
    for i in single_nodes:
        changed_data = np.concatenate((data[0:i, :], data[i + 1:, :]), axis=0) # delete correspondig row
        changed_data = np.concatenate((changed_data[:,0:i], changed_data[:, i + 1:]), axis=1) # delete correspondig column
    else:
      changed_data = data
    print(single_nodes)
    print('bst:', best_cluster)
    cluster = []
    if len(changed_data) != 0:
      if is_log:
        node_log = np.floor(np.log2(len(changed_data))).astype(int)
        row_num = node_log
        band_num = len(changed_data) // row_num
      else:
        row_num = ceil(len(changed_data) / 10)
        band_num = len(changed_data) // row_num
        
      # print('data',data, type(data))
      # score, cluster = LSH_simmilarity_matrix_permutation_arr(num_iter, changed_data, row_num, band_num)
      regular_LSH(num_iter, changed_data, row_num, band_num, 30)
    print(cluster, best_cluster)
    


    #merge two clusters
    j = 0 # index into cluster of changed_data
    for i in range(nodes): # loop over all nodes
       # if a node is not single meaning is not in the best cluster update it
       if i not in single_nodes: 
          best_cluster[i] = cluster[j]
          j += 1
    # print(best_cluster)
    score = fast_score(data, best_cluster)

    return score, best_cluster



            



# LSH

In [10]:
ds_name = 'new_'+str(1000)+'_rows.xlsx'
data = load_dataset(ds_name)

In [13]:
nodes = len(data)
node_log = np.floor(np.log2(len(data))).astype(int)
total_num_sigs=1000
num_iter = 20

row_num = node_log
band_num = total_num_sigs // row_num

score, cluster = regular_LSH(num_iter, data, row_num, band_num, total_num_sigs)
output_log = 'dataset: '+ ds_name +' \n row: '+ str(row_num) +' \n band: '+ str(band_num) +' \n score: '+ str(score) + ' \n number of signitures: ' +str(total_num_sigs)+'\n\n'
print(output_log)
# file.write(output_log)

  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
total_num_sigs=len(data)
j = total_num_sigs / 20
node_log = np.floor(np.log2(len(data))).astype(int)
row_num = node_log
band_num = total_num_sigs // row_num

score, cluster = LSH_simmilarity_matrix_permutation_arr(node_log * 5, data, row_num, band_num)
output_log = 'dataset: '+ ds_name +' \n row: '+ str(row_num) +' \n band: '+ str(band_num) +' \n score: '+ str(score) + ' \n number of signitures: ' +str(j)+'\n\n'
print(output_log)
# file.write(output_log)

NameError: name 'LSH_simmilarity_matrix_permutation_arr' is not defined

In [179]:
total_num_sigs=len(data)
j = total_num_sigs // 8
num_iter = 10
node_log = np.floor(np.log2(len(data))).astype(int)
row_num = node_log
band_num = total_num_sigs // row_num

score, cluster = LSH_singular_diff(num_iter, data, row_num, band_num, j, is_log = True)
output_log = 'dataset: '+ ds_name +' \n row: '+ str(row_num) +' \n band: '+ str(band_num) +' \n score: '+ str(score) + ' \n number of signitures: ' +str(j)+'\n\n'
print(output_log)
# file.write(output_log)

[3, 17, 27, 46, 47, 101, 110, 176, 232, 242]
bst: [0, 0, 0, 303, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 317, 0, 0, 0, 0, 0, 0, 0, 0, 0, 327, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 346, 347, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 401, 0, 0, 0, 0, 0, 0, 0, 0, 410, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 476, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 532, 0, 0, 0, 0, 0, 0, 0, 0, 0, 542, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


  0%|          | 0/10 [00:00<?, ?it/s]

IndexError: list index out of range